In [1]:
import torch

import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

map_location = torch.device('cuda:0')

In [2]:
import os
import shutil
import nip

path_to_config = '/home/msst/repo/Quantization/configs/vq/llama2_7b/symquant/llama2_symquant_cb256_vecdim8_C5e-5_L5e-5_lossL1_reassfrac1.0_3layershighbit.yaml'

config = nip.load(path_to_config)
logdir = config['logdir']
os.makedirs(logdir, exist_ok=True)
try:
    shutil.copy(path_to_config, logdir)
except shutil.SameFileError:
    pass


fp_blocks_path = config['fp_blocks_path']
q_blocks_path = config['q_blocks_path']
trained_q_blocks_path = os.path.join(config['logdir'], 'per_block_q_trained')

os.makedirs(trained_q_blocks_path, exist_ok=True)

In [3]:
trainer = qlib.HomequantTrainerPTQ(
    logdir = config['logdir'],
    training_config = config['training_params'],
    tokenizer=qlib.load_tokenizer(config['model_name']),
    rotary_emb=torch.load(f'{config["fp_blocks_path"]}/model.rotary_emb',),
	quant_classes=[qlib.SymHQLinear, qlib.HaarSymHQLinear],
	store_dtype=torch.float16
)

/tmp/ipykernel_225414/850949115.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rotary_emb=torch.load(f'{config["fp_blocks_path"]}/model.rotary_emb',),


Resolving data files:   0%|          | 0/200 [00:00<?, ?it/s]

In [4]:
print("model_decoder.embed_tokens")
trainer.finetune_block_ptq(
        path_to_fp_block = f'{fp_blocks_path}/model.embed_tokens',
        path_to_q_block = f'{q_blocks_path}/model.embed_tokens',
        path_to_q_block_trained = f'{trained_q_blocks_path}/model.embed_tokens',
        train=False,
        with_input_preparation=False,
    )

for layer_idx in range(32):
    print()
    print('layer_idx:', layer_idx)
        
    trainer.logger.block_label = f'block.{layer_idx}'
    # trainer.finetune_block_ptq(
    #     path_to_fp_block = f'{fp_blocks_path}/model.layers.{layer_idx}',
    #     path_to_q_block = f'{q_blocks_path}/model.layers.{layer_idx}',
    #     path_to_q_block_trained = f'{trained_q_blocks_path}/model.layers.{layer_idx}',
    #     train=True,
    #     with_input_preparation=True,
    # )
    trainer.finetune_block_ptq(
        path_to_fp_block = f'{fp_blocks_path}/model.layers.{layer_idx}',
        path_to_q_block = f'{q_blocks_path}/model.layers.{layer_idx}',
        #path_to_q_block = f'{q_blocks_path}/model.layers.{layer_idx}' if layer_idx>2 else f'{fp_blocks_path}/model.layers.{layer_idx}',
        path_to_q_block_trained = f'{trained_q_blocks_path}/model.layers.{layer_idx}',
        train=True if layer_idx >= 3 else False,
        with_input_preparation=True,
    )

model_decoder.embed_tokens


/home/msst/repo/Quantization/qlib/ptq/homequant_ptq_train.py:221: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  block = torch.load(path_to_fp_block, map_location=self.device


layer_idx: 0


collecting activations: 100%|██████████| 4/4 [00:01<00:00,  2.45it/s]



layer_idx: 1


...


layer_idx: 2


collecting activations: 100%|██████████| 4/4 [00:01<00:00,  2.47it/s]



layer_idx: 3


collecting activations: 100%|██████████| 4/4 [00:01<00:00,  2.62it/s]
/home/msst/repo/Quantization/qlib/ptq/homequant_ptq_utils.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental fea

KeyboardInterrupt: 